## Máquina de Turing

Creación de una Máquina de Turing de una cinta en Python.

In [1]:
from collections import defaultdict

class TuringMachine:
    """Simulador básico de Máquina de Turing de una cinta."""
    def __init__(self, tape_input, initial_state, final_states, transitions, blank_symbol='B'):
        """
        tape_input: Cinta inicial.
        initial_state: Estado inicial.
        final_states: Lista de estados en los cuales la máquina se detendrá.
        transitions: Diccionario de transiciones.
            Tiene la forma (estado_actual, simbolo_leido): (nuevo_simbolo, direccion, nuevo_estado)
            Puede utilizarse el símbolo '*' como wilcard para el simbolo_leido y nuevo_simbolo.
        blank_symbol: Símbolo blanco o espacio.
        """
        # La cinta se implementa con defaultdict para simular la infinitud
        self.tape = defaultdict(lambda: blank_symbol)
        for i, symbol in enumerate(tape_input):
            self.tape[i] = symbol
        
        self.head_position = 0
        self.current_state = initial_state
        self.final_states = set(final_states)
        self.transitions = transitions
        self.blank_symbol = blank_symbol
        # Agrego el estado inicial al historial
        self.history = [(self.current_state, self.get_tape_string(), self.head_position)]

    def step(self):
        """Ejecuta un paso de la MT."""
        if self.current_state in self.final_states:
            return False  # Se detiene
        
        current_symbol = self.tape[self.head_position]
        
        # La clave de la transición es (estado_actual, símbolo_leído)
        key = (self.current_state, current_symbol)
        # Si no hay transición para el símbolo actual, se prueba el wilcard
        # Con esto se busca reducir la cantidad de transiciones cuando 
        # se repiten para el mismo estado inicial y siguiente
        wilcard_key = (self.current_state, '*')
        
        t = self.transitions.get(key) or self.transitions.get(wilcard_key)
        if t:
            # La transición es (nuevo_estado, nuevo_símbolo, dirección)
            new_symbol, direction, new_state = t
            
            if new_symbol == '*':
                # Si el nuevo símbolo es un wilcard, se mantiene el símbolo actual
                new_symbol = current_symbol

            # 1. Escritura
            self.tape[self.head_position] = new_symbol
            
            # 2. Movimiento del cabezal
            if direction == 'R':
                self.head_position += 1
            elif direction == 'L':
                self.head_position -= 1
            elif direction == 'S':
                # 'S' para Stationary (Quieto)
                pass
            else:
                raise ValueError(f"Dirección inválida: {direction}")
                
            # 3. Cambio de estado
            self.current_state = new_state
            
            # Opcional: registrar el estado actual
            self.history.append((self.current_state, self.get_tape_string(), self.head_position))
            
            return True  # Continuar
        else:
            # No hay transición definida (detención implícita por error o final)
            raise ValueError(f"Transición no definida para el estado {self.current_state} y el símbolo {current_symbol}")
            # return False

    def run(self, max_steps=1000):
        """Ejecuta la MT hasta que se detenga o alcance el máximo de pasos."""
        steps = 0
        while self.step() and steps < max_steps:
            steps += 1
        
        if (steps >= max_steps):
            raise ValueError("Superado el máximo de pasos")

        # Retorna el resultado (el contenido de la cinta)
        return self.get_tape_string()

    def get_tape_string(self):
        """Obtiene la porción relevante de la cinta como una cadena."""
        # Encontrar los límites de la parte utilizada de la cinta
        indices = [i for i, symbol in self.tape.items() if symbol != self.blank_symbol]
        if not indices:
            return self.blank_symbol
            
        min_idx = min(indices)
        max_idx = max(indices)
        
        # Construir la cadena de la cinta dentro de los límites        
        return "".join(self.tape[i] for i in range(min_idx, max_idx + 1))

In [2]:
# Programa para simular una Máquina de Turing que divide dos números binarios.
def division(dividendo, divisor, show_steps = False):
    # Conversión de números enteros a binarios
    bin_dividendo = bin(dividendo)[2:]
    bin_divisor = bin(divisor)[2:]

    # Definición del símbolo blanco
    blank_symbol = "B"

    # Cantidad máxima de dígitos entre el dividendo y el divisor.
    # Es un requisito para que la transición de estados de este programa
    # funcione correctamente.
    max_digits = max(len(bin_dividendo), len(bin_divisor))
    # Se agregan ceros a la izquierda para que ambos números tengan la misma longitud.
    # Cinta con estado inicial.
    tape_input = f"{bin_dividendo.zfill(max_digits)}{blank_symbol}{bin_divisor.zfill(max_digits)}"

    # Estado inicial y estado final para este programa.
    initial_state = "q00"
    final_states = {"qha"}

    # Transiciones para este programa.
    # Se utilizó la forma de escritura con comodines para reducir la tabla de transiciones.
    # Las transiciones se definen como:
    #  (estado_actual, simbolo_leido): (nuevo_simbolo, direccion, nuevo_estado)
    transitions = {
        ("q00","*"): ("*","R","q00"),
        ("q00","B"): ("B","R","q01"),
        ("q01","1"): ("u","R","q02"),
        ("q01","0"): ("v","R","q06"),
        ("q01","#"): ("#","L","q08"),
        ("q02","*"): ("*","R","q02"),
        ("q02","B"): ("#","R","q03"),
        ("q02","#"): ("#","R","q03"),
        ("q03","*"): ("*","R","q03"),
        ("q03","B"): ("1","L","q04"),
        ("q04","*"): ("*","L","q04"),
        ("q04","#"): ("#","L","q05"),
        ("q05","*"): ("*","L","q05"),
        ("q05","u"): ("u","R","q01"),
        ("q05","v"): ("v","R","q01"),
        ("q06","*"): ("*","R","q06"),
        ("q06","B"): ("#","R","q07"),
        ("q06","#"): ("#","R","q07"),
        ("q07","*"): ("*","R","q07"),
        ("q07","B"): ("0","L","q04"),
        ("q08","v"): ("0","L","q08"),
        ("q08","u"): ("1","L","q09"),
        ("q09","u"): ("0","L","q09"),
        ("q09","v"): ("1","L","q09"),
        ("q09","B"): ("B","L","q10"),
        ("q10","*"): ("*","L","q10"),
        ("q10","B"): ("B","R","q11"),
        ("q11","*"): ("*","R","q11"),
        ("q11","B"): ("B","R","q12"),
        ("q12","*"): ("*","R","q12"),
        ("q12","#"): ("#","L","q13"),
        ("q13","0"): ("u","L","q14"),
        ("q13","B"): ("B","R","q19"),
        ("q13","1"): ("v","L","q15"),
        ("q13","v"): ("v","L","q13"),
        ("q13","u"): ("u","L","q13"),
        ("q14","*"): ("*","L","q14"),
        ("q14","B"): ("B","L","q36"),
        ("q36","*"): ("*","L","q36"),
        ("q36","B"): ("B","R","q11"),
        ("q36","1"): ("y","R","q11"),
        ("q36","0"): ("x","R","q11"),
        ("q19","u"): ("0","R","q19"),
        ("q19","v"): ("1","R","q19"),
        ("q19","#"): ("#","L","q37"),
        ("q20","x"): ("0","L","q20"),
        ("q20","y"): ("1","L","q20"),
        ("q20","*"): ("*","L","q20"),
        ("q20","B"): ("B","L","q39"),
        ("q39","1"): ("0","L","q39"),
        ("q39","B"): ("1","R","q40"),
        ("q39","0"): ("1","R","q40"),
        ("q40","*"): ("*","R","q40"),
        ("q40","B"): ("B","R","q41"),
        ("q41","B"): ("B","R","q41"),
        ("q41","*"): ("B","R","q22"),
        ("q15","*"): ("*","L","q15"),
        ("q15","B"): ("B","L","q16"),
        ("q16","*"): ("*","L","q16"),
        ("q16","1"): ("0","L","q16"),
        ("q16","B"): ("1","S","q17"),
        ("q16","0"): ("1","S","q17"),
        ("q17","*"): ("*","R","q17"),
        ("q17","B"): ("B","L","q18"),
        ("q17","x"): ("x","L","q18"),
        ("q17","y"): ("y","L","q18"),
        ("q18","1"): ("y","R","q11"),
        ("q18","0"): ("x","R","q11"),
        ("q37","*"): ("*","L","q37"),
        ("q37","B"): ("B","L","q20"),
        ("q22","B"): ("B","R","q35"),
        ("q22","1"): ("y","R","q23"),
        ("q22","0"): ("x","R","q27"),
        ("q23","*"): ("*","R","q23"),
        ("q23","#"): ("#","R","q24"),
        ("q24","*"): ("*","R","q24"),
        ("q24","1"): ("y","L","q25"),
        ("q24","B"): ("B","L","q29"),
        ("q24","0"): ("0","L","q29"),
        ("q25","*"): ("*","L","q25"),
        ("q25","B"): ("B","L","q26"),
        ("q26","*"): ("*","L","q26"),
        ("q26","x"): ("x","R","q22"),
        ("q26","y"): ("y","R","q22"),
        ("q27","*"): ("*","R","q27"),
        ("q27","#"): ("#","R","q28"),
        ("q28","*"): ("*","R","q28"),
        ("q28","0"): ("x","L","q25"),
        ("q28","B"): ("B","L","q29"),
        ("q28","1"): ("1","R","q32"),
        ("q29","x"): ("0","L","q29"),
        ("q29","y"): ("1","L","q29"),
        ("q29","#"): ("#","L","q30"),
        ("q30","*"): ("*","L","q30"),
        ("q30","B"): ("B","L","q31"),
        ("q31","x"): ("0","L","q31"),
        ("q31","y"): ("1","L","q31"),
        ("q31","B"): ("B","R","q11"),
        ("q31","0"): ("0","L","q31"),
        ("q31","1"): ("1","L","q31"),
        ("q32","*"): ("*","R","q32"),
        ("q32","B"): ("B","L","q33"),
        ("q33","*"): ("B","L","q33"),
        ("q33","B"): ("B","L","q34"),
        ("q34","x"): ("0","L","q34"),
        ("q34","y"): ("1","L","q34"),
        ("q34","B"): ("B","R","qha"),
        ("q34","*"): ("*","L","q34"),
        ("q35","*"): ("*","R","q35"),
        ("q35","B"): ("B","L","q29"),
        }

    # Inicialización de la Máquina de Turing
    tm = TuringMachine(
        tape_input=tape_input,
        initial_state=initial_state,
        final_states=final_states,
        transitions=transitions,
        blank_symbol=blank_symbol
    )

    # Ejecución de la Máquina de Turing con el programa de división binaria.
    result = tm.run(max_steps=10000)
    # Procesamiento del resultado. Separación y conversión a decimal.
    tmp_result =  [item for item in result.split(blank_symbol) if item]
    cociente = int(tmp_result[0], 2)
    resto = int(tmp_result[1], 2)
    print(f"El resultado de dividir {dividendo} ({bin_dividendo.lstrip('0') or '0'}) por {divisor} ({bin_divisor.lstrip('0') or '0'}) es:\n",
           f"\t{cociente} ({tmp_result[0].lstrip('0') or '0'}) con resto {resto} ({tmp_result[1].lstrip('0') or '0'}).")
    if show_steps:
        print(f"Pasos realizados {len(tm.history)-1}:")
        print("(estado_actual, cinta, posición_cabezal)")
        for step in tm.history:
            print(step)


In [3]:
division(5, 2, show_steps=True)

El resultado de dividir 5 (101) por 2 (10) es:
 	2 (10) con resto 1 (1).
Pasos realizados 282:
(estado_actual, cinta, posición_cabezal)
('q00', '101B010', 0)
('q00', '101B010', 1)
('q00', '101B010', 2)
('q00', '101B010', 3)
('q01', '101B010', 4)
('q06', '101Bv10', 5)
('q06', '101Bv10', 6)
('q06', '101Bv10', 7)
('q07', '101Bv10#', 8)
('q04', '101Bv10#0', 7)
('q05', '101Bv10#0', 6)
('q05', '101Bv10#0', 5)
('q05', '101Bv10#0', 4)
('q01', '101Bv10#0', 5)
('q02', '101Bvu0#0', 6)
('q02', '101Bvu0#0', 7)
('q03', '101Bvu0#0', 8)
('q03', '101Bvu0#0', 9)
('q04', '101Bvu0#01', 8)
('q04', '101Bvu0#01', 7)
('q05', '101Bvu0#01', 6)
('q05', '101Bvu0#01', 5)
('q01', '101Bvu0#01', 6)
('q06', '101Bvuv#01', 7)
('q07', '101Bvuv#01', 8)
('q07', '101Bvuv#01', 9)
('q07', '101Bvuv#01', 10)
('q04', '101Bvuv#010', 9)
('q04', '101Bvuv#010', 8)
('q04', '101Bvuv#010', 7)
('q05', '101Bvuv#010', 6)
('q01', '101Bvuv#010', 7)
('q08', '101Bvuv#010', 6)
('q08', '101Bvu0#010', 5)
('q09', '101Bv10#010', 4)
('q09', '101B11

In [4]:
division(15, 3)
division(15, 7)
division(15, 9)

El resultado de dividir 15 (1111) por 3 (11) es:
 	5 (101) con resto 0 (0).
El resultado de dividir 15 (1111) por 7 (111) es:
 	2 (10) con resto 1 (1).
El resultado de dividir 15 (1111) por 9 (1001) es:
 	1 (1) con resto 6 (110).


In [5]:
division(31, 3)
division(31, 4)
division(31, 5)
division(31, 6)
division(31, 7)
division(31, 8)
division(31, 9)

El resultado de dividir 31 (11111) por 3 (11) es:
 	10 (1010) con resto 1 (1).
El resultado de dividir 31 (11111) por 4 (100) es:
 	7 (111) con resto 3 (11).
El resultado de dividir 31 (11111) por 5 (101) es:
 	6 (110) con resto 1 (1).
El resultado de dividir 31 (11111) por 6 (110) es:
 	5 (101) con resto 1 (1).
El resultado de dividir 31 (11111) por 7 (111) es:
 	4 (100) con resto 3 (11).
El resultado de dividir 31 (11111) por 8 (1000) es:
 	3 (11) con resto 7 (111).
El resultado de dividir 31 (11111) por 9 (1001) es:
 	3 (11) con resto 4 (100).


Se utilizó como referencia para la implementación de los estados de la Máquina de Turing el documento <a href="documentos/BINARY DIVISION TURING MACHINE.pdf">BINARY DIVISION TURING MACHINE.pdf</a> que se encuentra en el repositorio de GitHub https://github.com/Rishabhkandoi/Binary-Division-Turing-Machine/blob/master/TOC_FINAL.pdf


## Comprobación del funcionamiento de la Máquina de Turing:  

División 5 (101) por 2 (010)

Initial input: 101 010  
Ejemplo en línea: https://morphett.info/turing/turing.html?8166a68430deff31c50be02c44672188  
Final tape: 10  001

Imagen del estado final:


<img src="imagenes/Verificación división binaria en máquina de Turing.png">